<a href="https://colab.research.google.com/github/Cami050/proyecto_exploracion_datos/blob/main/Proyecto_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar gdown (un gestor de paquetes de Python)si aún no está instalado
!pip install gdown -q